In [41]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [42]:
import taichi as ti
ti.init(arch=ti.cpu)

[Taichi] Starting on arch=arm64


In [43]:
# Параметры системы
N = 50  # Число осцилляторов по горизонтали
M = 50  # Число осцилляторов по вертикали
m = 0.1  # Масса каждого осциллятора
g = 9.8  # Ускорение свободного падения
dt = 0.01  # Шаг времени
nstep = 30  # Количество шагов

# Параметры связей
l0 = 1/(N-1)  # Длина нерастяжимой связи
Cs = 500.0  # Жесткость при контакте со сферой
Bs = 5.0  # Коэффициент демпфирования при контакте со сферой
R = 0.25  # Радиус сферы
sphere_center = ti.Vector([1.0, 1.0, 1.0])  # Центр сферы


In [44]:
l0

0.02040816326530612

In [45]:
# Начальные условия
x = ti.Vector.field(3, dtype=ti.f32)
v = ti.Vector.field(3, dtype=ti.f32)
a = ti.Vector.field(3, dtype=ti.f32)
ti.root.pointer(ti.i, N).dense(ti.j, M).place(x)
ti.root.pointer(ti.i, N).dense(ti.j, M).place(v)
ti.root.pointer(ti.i, N).dense(ti.j, M).place(a)
forces = ti.Vector.field(3, dtype=ti.f32)
ti.root.pointer(ti.i, N).dense(ti.j, M).place(forces)
x_prev = ti.Vector.field(3, dtype=ti.f32)
ti.root.pointer(ti.i, N).dense(ti.j, M).place(x_prev)
# Расположение осцилляторов в начальный момент
@ti.kernel
def init_x_v():
    for i in range(N):
        for j in range(M):
            x[i, j] = [i * l0 + 0.5, j * l0 + 0.5, 1.5]  # Начальная высота 1.5
            v[i, j] = [0, 0, 0]
            v[i, j] = [0, 0, 0]
            # Предыдущие положения для метода Верле
            x_prev[i, j] = x[i, j] - v[i, j] * dt + 0.5 * a[i, j] * dt**2

In [46]:
init_x_v()

In [47]:
x.to_numpy()

array([[[0.5       , 0.5       , 1.5       ],
        [0.5       , 0.52040815, 1.5       ],
        [0.5       , 0.5408163 , 1.5       ],
        ...,
        [0.5       , 1.4591837 , 1.5       ],
        [0.5       , 1.4795918 , 1.5       ],
        [0.5       , 1.5       , 1.5       ]],

       [[0.52040815, 0.5       , 1.5       ],
        [0.52040815, 0.52040815, 1.5       ],
        [0.52040815, 0.5408163 , 1.5       ],
        ...,
        [0.52040815, 1.4591837 , 1.5       ],
        [0.52040815, 1.4795918 , 1.5       ],
        [0.52040815, 1.5       , 1.5       ]],

       [[0.5408163 , 0.5       , 1.5       ],
        [0.5408163 , 0.52040815, 1.5       ],
        [0.5408163 , 0.5408163 , 1.5       ],
        ...,
        [0.5408163 , 1.4591837 , 1.5       ],
        [0.5408163 , 1.4795918 , 1.5       ],
        [0.5408163 , 1.5       , 1.5       ]],

       ...,

       [[1.4591837 , 0.5       , 1.5       ],
        [1.4591837 , 0.52040815, 1.5       ],
        [1.4591837 , 0

In [48]:
# Функция для расчета сил
@ti.kernel
def compute_forces():
    # Гравитация
    for i in range(N):
        for j in range(M):
            forces[i, j] = [0, 0, - m * g]
    # Взаимодействие со сферой
    for i in range(N):
        for j in range(M):
            r = x[i, j] - sphere_center
            # dist = np.linalg.norm(r)
            dist = ti.sqrt(r.x ** 2 + r.y ** 2 + r.z ** 2)
            if dist < R:  # Если масса внутри сферы
                normal = r / dist  # Нормаль к поверхности сферы
                penetration = R - dist  # Глубина проникновения
                # relative_velocity = np.dot(v[i, j], normal)  # Проекция скорости на нормаль
                relative_velocity = v[i,j] * normal
                # Добавляем силу отталкивания и демпфирования
                forces[i, j] += Cs * penetration * normal + Bs * relative_velocity * normal
                # Корректируем положение массы на поверхности сферы
                for _ in range(5):  # Итеративная коррекция
                    r = x[i, j] - sphere_center
                    # dist = np.linalg.norm(r)
                    dist = ti.sqrt(r.x ** 2 + r.y ** 2 + r.z ** 2)
                    if dist < R:
                        normal = r / dist
                        penetration = R - dist
                        x[i, j] += penetration * normal
                # Разделяем скорость на нормальную и тангенциальную составляющие
                normal = r / dist
                # v_normal = np.dot(v[i, j], normal) * normal
                v_normal = v[i, j].dot(normal) * normal
                v_tangent = v[i, j] - v_normal
                v[i, j] = v_tangent  # Сохраняем только тангенциальную составляющую
                # Проверка, что масса на поверхности
                # assert np.isclose(np.linalg.norm(x[i, j] - sphere_center), R), \
                #     f"Ошибка: масса не на поверхности сферы, dist={np.linalg.norm(x[i, j] - sphere_center)}"
    # return forces

In [49]:
# Проекционный метод для восстановления нерастяжимых связей。PD
@ti.kernel
def enforce_constraints():
    tolerance = 1e-6  # Допустимая погрешность
    max_iterations = 100  # Максимальное число итераций
    for _ in range(max_iterations):
        max_error = 0.0
        for i in range(N):
            for j in range(M):
                forces[i, j] = [0, 0, - m * g]
                # Коррекция связей с соседями
                if i < N - 1:  # Горизонтальная связь 
                    dx = x[i + 1, j] - x[i, j]
                    dist = ti.sqrt(dx.x ** 2 + dx.y ** 2 + dx.z ** 2)
                    error = abs(dist - l0)
                    max_error = max(max_error, error)
                    correction = (l0 - dist) / (2 * dist) * dx
                    x[i + 1, j] += correction
                    x[i, j] -= correction
                if j < M - 1:  # Вертикальная связь
                    dy = x[i, j + 1] - x[i, j]
                    # dist = np.linalg.norm(dy)
                    dist = ti.sqrt(dy.x ** 2 + dy.y ** 2 + dy.z ** 2)
                    error = abs(dist - l0)
                    max_error = max(max_error, error)
                    correction = (l0 - dist) / (2 * dist) * dy
                    x[i, j + 1] += correction
                    x[i, j] -= correction
        if max_error < tolerance:
            break

In [50]:
# Функция для визуализации текущего состояния
def visualize_step(x, step):
    xs, ys, zs = x[:, :, 0], x[:, :, 1], x[:, :, 2]
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))  # Четыре панели (2x2)
    # Общий трехмерный вид
    ax_general = fig.add_subplot(2, 2, 1, projection='3d')
    ax_general.scatter(xs, ys, zs, c='blue', s=50, label='Массы')
    # Отображение связей
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_general.plot([x[i, j, 0], x[i + 1, j, 0]],
                                [x[i, j, 1], x[i + 1, j, 1]],
                                [x[i, j, 2], x[i + 1, j, 2]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_general.plot([x[i, j, 0], x[i, j + 1, 0]],
                                [x[i, j, 1], x[i, j + 1, 1]],
                                [x[i, j, 2], x[i, j + 1, 2]], color='gray', linewidth=0.5)
    # Отображение сферы
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 50)
    x_sphere = sphere_center[0] + R * np.outer(np.cos(u), np.sin(v))
    y_sphere = sphere_center[1] + R * np.outer(np.sin(u), np.sin(v))
    z_sphere = sphere_center[2] + R * np.outer(np.ones(np.size(u)), np.cos(v))
    ax_general.plot_surface(x_sphere, y_sphere, z_sphere, color='red', alpha=0.5, label='Сфера')
    # Настройка графика
    ax_general.set_xlim(-0.1, 2.0)
    ax_general.set_ylim(-0.1, 2.0)
    ax_general.set_zlim(-0.1, 2.0)
    ax_general.set_box_aspect([1, 1, 1])
    ax_general.set_xlabel('X')
    ax_general.set_ylabel('Y')
    ax_general.set_zlabel('Z')
    ax_general.set_title("Общий вид")
    ax_general.legend()
    # Вид сверху вдоль оси Z
    ax_top = fig.add_subplot(2, 2, 2)
    ax_top.scatter(xs, ys, c='blue', s=50, label='Массы')
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_top.plot([x[i, j, 0], x[i + 1, j, 0]],
                            [x[i, j, 1], x[i + 1, j, 1]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_top.plot([x[i, j, 0], x[i, j + 1, 0]],
                            [x[i, j, 1], x[i, j + 1, 1]], color='gray', linewidth=0.5)
    ax_top.set_xlim(-0.1, 2.0)
    ax_top.set_ylim(-0.1, 2.0)
    ax_top.set_xlabel('X')
    ax_top.set_ylabel('Y')
    ax_top.set_title("Вид сверху (по Z)")
    ax_top.legend()
    # Вид сбоку вдоль оси X
    ax_side_x = fig.add_subplot(2, 2, 3)
    ax_side_x.scatter(ys, zs, c='blue', s=50, label='Массы')
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_side_x.plot([x[i, j, 1], x[i + 1, j, 1]],
                               [x[i, j, 2], x[i + 1, j, 2]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_side_x.plot([x[i, j, 1], x[i, j + 1, 1]],
                               [x[i, j, 2], x[i, j + 1, 2]], color='gray', linewidth=0.5)
    ax_side_x.set_xlim(-0.1, 2.0)
    ax_side_x.set_ylim(-0.1, 2.0)
    ax_side_x.set_xlabel('Y')
    ax_side_x.set_ylabel('Z')
    ax_side_x.set_title("Вид сбоку (по X)")
    ax_side_x.legend()
    # Вид сбоку вдоль оси Y
    ax_side_y = fig.add_subplot(2, 2, 4)
    ax_side_y.scatter(xs, zs, c='blue', s=50, label='Массы')
    for i in range(N):
        for j in range(M):
            if i < N - 1:  # Горизонтальные связи
                ax_side_y.plot([x[i, j, 0], x[i + 1, j, 0]],
                               [x[i, j, 2], x[i + 1, j, 2]], color='gray', linewidth=0.5)
            if j < M - 1:  # Вертикальные связи
                ax_side_y.plot([x[i, j, 0], x[i, j + 1, 0]],
                               [x[i, j, 2], x[i, j + 1, 2]], color='gray', linewidth=0.5)
    ax_side_y.set_xlim(-0.1, 2.0)
    ax_side_y.set_ylim(-0.1, 2.0)
    ax_side_y.set_xlabel('X')
    ax_side_y.set_ylabel('Z')
    ax_side_y.set_title("Вид сбоку (по Y)")
    ax_side_y.legend()
    # Сохранение кадра
    frame_path = os.path.join(output_dir, f"frame_{step:04d}.png")
    plt.tight_layout()
    plt.savefig(frame_path)
    plt.close(fig)

In [51]:
@ti.kernel
def update_x_v():
    for i in range(N):
        for j in range(M):
            forces[i, j] = [0, 0, - m * g]
            a[i, j] = forces[i, j] / m  # Вычисляем ускорения
            # Метод Верле
            x_next = 2 * x[i, j] - x_prev[i, j] + a[i, j] * dt**2
            v_new = (x_next - x_prev[i, j]) / (2 * dt)
            x_prev[i, j] = x[i, j]
            x[i, j] = x_next
            v[i, j] = v_new

In [52]:
# Создание каталога для сохранения кадров
output_dir = f"frames_{N}*{M}_taichi"
os.makedirs(output_dir, exist_ok=True)

# Моделирование
positions = []
for step in range(nstep):
    print(f"STEP: {step}")
    compute_forces()  # Рассчитываем силы
    update_x_v()
    # Применяем проекционный метод
    enforce_constraints()
    positions.append(x.to_numpy().copy())
    # print(f"X {x.to_numpy()}")
    # Визуализация
    visualize_step(x.to_numpy(), step)
        
print(f"Моделирование завершено. Кадры сохранены в каталог '{output_dir}'.")

STEP: 0
STEP: 1
STEP: 2


/var/folders/wg/vzdjz4_n2q3gtl63x660zg2r0000gn/T/ipykernel_71905/3872868707.py:90: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(frame_path)


STEP: 3
STEP: 4
STEP: 5
STEP: 6
STEP: 7
STEP: 8
STEP: 9
STEP: 10
STEP: 11
STEP: 12
STEP: 13
STEP: 14
STEP: 15
STEP: 16
STEP: 17
STEP: 18
STEP: 19
STEP: 20
STEP: 21
STEP: 22


/var/folders/wg/vzdjz4_n2q3gtl63x660zg2r0000gn/T/ipykernel_71905/3872868707.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


STEP: 23
STEP: 24
STEP: 25
STEP: 26
STEP: 27
STEP: 28
STEP: 29
Моделирование завершено. Кадры сохранены в каталог 'frames_50*50_taichi'.


In [53]:
import imageio.v2 as imageio
import os

# Список файлов кадров
frame_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".png")])

# Создание анимации
with imageio.get_writer(f"animation{N}_taichi.gif", mode="I", duration=0.05) as writer:
    for frame_file in frame_files:
        image = imageio.imread(frame_file)
        writer.append_data(image)

print(f"Анимация сохранена как 'animation{N}_taichi.gif'.")

Анимация сохранена как 'animation50_taichi.gif'.
